In [ ]:
# Soil moisture plots over time

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import seaborn as sns
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
import matplotlib.dates as mdates

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_soil         = Read_Loobos_soil       (years,datapath) 
    progress.append('dataloaded')

In [ ]:
#the soil dataset is comprised of two parts.
#before 2005, soil moisture is measured using a different sensor but data is missing (only codes 202 and 204 have data)
#after 2005, soil moisture is measured using SM-sensor
df_soil.columns

In [ ]:
SM_data=df_soil.loc['2005-04-11':,['SM-003','SM-020','SM-050','SM-100']] #measured data for SM starts on 11-04-2005 at around 14:00
SM_data

#### EC series EC202 and EC204

In [ ]:
#all EC series empty besides 202 and 204
df_soil['EC202'].dropna().plot()
df_soil['EC204'].dropna().plot()

#### Eps_co series 

In [ ]:
# All Eps#co series empty besides 202 and 204
#df_soil['Eps201co'].dropna().plot()
df_soil['Eps202co'].dropna().plot()
#df_soil['Eps203co'].dropna().plot()
df_soil['Eps204co'].dropna().plot()
#df_soil['Eps205co'].dropna().plot()
#df_soil['Eps206co'].dropna().plot()
#df_soil['Eps207co'].dropna().plot()
#df_soil['Eps208co'].dropna().plot()
#df_soil['Eps209co'].dropna().plot()

### end of EC and Eps_co series plots

# Construct soil moisture layers

In [ ]:
# based on a method in a paper, they separate soil moist into 2 layers:
# 0-50 and 50-100 (littoral excluded). They do this because they want to plot C3 grass and pinus sylvestris.
# in theory, we could do 1 big soil layer, as we do not plot GPP for grass (unlike the paper)
# however, the influence of the GPP contribution from understory vegetation on observations should be there
# and given that grasses and black cherry both only should root to 50cm, these are useful depths to distinguish.

In [ ]:
SM_data['SM_l1']=SM_data.loc[:,['SM-003','SM-020','SM-050']].mean(axis='columns')
SM_data['SM_l2']=SM_data.loc[:,['SM-050','SM-100']].mean(axis='columns')
SM_data
#.loc['2005-01-02':'2005-01-14','SM-003'].plot()

In [ ]:
SM_data['SM_l1'].plot()
SM_data['SM_l2'].plot()

In [ ]:
fig,ax= plt.subplots()
ax.hist(SM_data['SM_l1'],bins=12)
ax.set_xlabel('vol soil moisture content (%)')
ax.set_xlabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.histplot(data=SM_data, x="SM_l1",ax=ax)
ax.set_xlim(0,0.1)
plt.show()

In [ ]:
SM_data

In [ ]:
#print min, max of each layer
print('003',min(SM_data['SM-003'].dropna()),' ',max(SM_data['SM-003'].dropna()))
print('020',min(SM_data['SM-020'].dropna()),' ',max(SM_data['SM-020'].dropna()))
print('050',min(SM_data['SM-050'].dropna()),' ',max(SM_data['SM-050'].dropna()))
print('100',min(SM_data['SM-100'].dropna()),' ',max(SM_data['SM-100'].dropna()))
print('sm1',min(SM_data['SM_l1'].dropna()),' ',max(SM_data['SM_l1'].dropna()))
print('sm2',min(SM_data['SM_l2'].dropna()),' ',max(SM_data['SM_l2'].dropna()))

## export data to csv

In [ ]:
SM_data.index.name = 'datetime'
SM_data.to_csv('Soil_moisture.csv')

## end of export data

## behavior of soil moist during heatwaves

In [ ]:
#def the plotter wrapper
plt.rcParams["figure.figsize"] = (20,6)

def SM_plot_wrapper(data,st,ed,tick_frq='Day'):
    fig,ax = plt.subplots()
    

    if tick_frq=='Day':
        ax.xaxis.set_minor_locator(mdates.DayLocator())
    elif tick_frq=='Hour':
        ax.xaxis.set_minor_locator(mdates.HourLocator())


    ax.plot(data.loc[st:ed,'SM_l1'],label='SM_layer1(0-50cm)')
    ax.plot(data.loc[st:ed,'SM_l2'],label='SM_layer2(50-100cm)')
    
    ax.set_ylabel('soil moist')
    
    ax.legend(loc=("upper left"))
    fig.autofmt_xdate()

    plt.show()

In [ ]:
#import the heatwave data
heatwave=pd.read_csv('Heatwave_table.csv')
heatwave.loc[:,'start']        = pd.to_datetime(heatwave['start'])
heatwave.loc[:,'end(inc.)']    = pd.to_datetime(heatwave['end(inc.)'])
heatwave.loc[:,'MaxTempDate']  = pd.to_datetime(heatwave['MaxTempDate'])
heatwave.index=heatwave['start']
heatwave

In [ ]:
dates=heatwave.loc['2005-04-11':'2019-08-28',['start','end(inc.)']]
dates

In [ ]:
#plot heatwaves
#print(test)
for index,row in dates.iterrows():
    SM_plot_wrapper(SM_data,row['start'],row['end(inc.)'],'Hour') #args: data, st,ed,[tick_frq='Day' or 'Hour']

## end of behavior soil moist during heatwaves